In [ ]:
# Jupyter Notebook for Model Training

import os
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Function to load the dataset
def load_dataset(annotations_path, images_folder):
    annotations = pd.read_csv(annotations_path)
    image_paths = [os.path.join(images_folder, f"{row['image_id']}.jpg") for _, row in annotations.iterrows()]
    return image_paths, annotations

# Function to preprocess the dataset
def preprocess_dataset(image_paths, annotations, label_map_path):
    label_map = label_map_util.load_labelmap(label_map_path)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    annotations['class_int'] = annotations['class'].apply(lambda x: next(item['id'] for item in category_index if item['name'] == x))

    return annotations

# Function to create a TensorFlow Dataset
def create_tf_dataset(image_paths, annotations):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, annotations))
    dataset = dataset.map(load_image_and_annotations, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

# Function to load an image and its annotations
def load_image_and_annotations(image_path, annotations):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))  # Adjust the size as needed

    # Preprocess annotations, convert to TensorFlow compatible format
    # ...

    return image, annotations

# Function to create and compile the detection model
def create_detection_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Paths
annotations_path = '
images_folder = '
label_map_path = '

# Load and preprocess the dataset
image_paths, annotations = load_dataset(annotations_path, images_folder)
annotations = preprocess_dataset(image_paths, annotations, label_map_path)

# Create TensorFlow Dataset
batch_size = 32
train_dataset = create_tf_dataset(image_paths, annotations).batch(batch_size)

# Create and compile the detection model
num_classes = 1  # Adjust based on your dataset
detection_model = create_detection_model(num_classes)

# Train the model
epochs = 10  # Adjust as needed
detection_model.fit(train_dataset, epochs=epochs)

# Save the trained model
output_model_path = '/models/detection_model.h5'
tf.saved_model.save(detection_model, output_model_path)

print("Model training completed.")
